# Duck Typing, Interfaces, and Benchmarkin

Julia's 'interfaces' and duck-typing are a core part of Julia's type system.

> A lot of the power and extensibility in Julia comes from a collection of informal interfaces. By extending a few specific methods to work for a custom type, objects of that type not only receive those functionalities, but they are also able to be used in other methods that are written to generically build upon those behaviors.

## Example 1: `UnitRange`

In [ ]:
x = 1:30

In [ ]:
typeof(x)

In [ ]:
supertypes(UnitRange)

In [ ]:
typeof(x) <: AbstractArray

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [ ]:
x[3]

In [ ]:
size(x)

In [ ]:
eltype(x)

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [ ]:
fieldnames(typeof(x))

or just by inspecting the source code

In [ ]:
@which UnitRange{Int64}(1, 30)

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Allocating memory is typically costly.

In [ ]:
@time collect(1:10_000_000);

But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [ ]:
@time 1:10_000_000;

This is so far the same as in Python, but here is a key difference:

```ipython
In [1]: a = range(10000)

In [2]: a
Out[2]: range(0, 10000)

In [3]: a * 2
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[3], line 1
----> 1 a * 2

TypeError: unsupported operand type(s) for *: 'range' and 'int'
```

As Julia uses multiple dispatch, `UnitRange` has its own methods defined for arithmetic which are **custom made for that type**. Not only does it behave the same way as an array with the same operations, it also does so **with great performance**:

In [ ]:
# Double all elements in the range and print off the 10th element:
@time r = collect(1:10_000_000) * 2
@time r = (1:10_000_000) * 2
println(r[10])

In [ ]:
# Sum all of the elements in the range:
@time sum(collect(1:10_000_000))
@time sum(1:10_000_000)

Parts of this do work in Python:

```ipython
In [1]: %time sum(range(10_000_000))
CPU times: user 183 ms, sys: 169 µs, total: 183 ms
Wall time: 183 ms
```

This effectively does the same thing the `collect` example does, just a few times slower, and far slower than the example using a range.

In [ ]:
@which sum(1:10_000_000)

Duck typing and interfaces let us define specialised code that is very performant, and interacts with the rest of the language as expected.

### Example 2: `DiagonalMatrix`

Let's create a simple custom `DiagonalMatrix` type that can represent square diagonal matrices, i.e.

$$ D = \left( \begin{matrix} x & 0 & 0 & 0 \\ 0 & y & 0 & 0 \\ 0 & 0 & z & 0 \\ 0 & 0 & 0 & \ddots \end{matrix} \right) $$

In [ ]:
struct DiagonalMatrix{T} <: AbstractArray{T,2}
    diag::AbstractVector{T}
end

In the spirit of duck typing, we integrate our `DiagonalMatrix` into Julia's type hierarchy by making it a subtype (`<:`) of `AbstractMatrix` to indicate **array-like behavior**. (Note that this does not indicate inheritence of structure!)

Of course, to actually make it behave like a matrix (a two-dimensional array) we must also implement (parts of) the [`AbstractArray` interface](https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1).

In [ ]:
DiagonalMatrix([1, 2, 3])

In [ ]:
Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

In [ ]:
DiagonalMatrix([1, 2, 3])

In [ ]:
function Base.getindex(D::DiagonalMatrix, i::Int, j::Int)
    if i == j
        return D.diag[i]
    else
        return zero(eltype(D))
    end
end

In [ ]:
D = DiagonalMatrix([1, 2, 3])

Note how it's automagically pretty printed (despite the fact that we never defined any special printing)!

In [ ]:
D[2, 2]

In [ ]:
D[1, 2]

In [ ]:
size(D)

In [ ]:
D[3, 3] = 5

In [ ]:
function Base.setindex!(D::DiagonalMatrix, v, i::Int, j::Int)
    if i == j
        D.diag[i] = v
    else
        throw(ArgumentError("cannot set off-diagonal entry ($i, $j)"))
    end
    return v
end

Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

In [ ]:
D[3, 3] = 5

In [ ]:
D

In [ ]:
D[3, 4] = 5

But that's not it. Because of duck typing, all kinds of different functions now "just work".

In [ ]:
eltype(D) # element data type

In [ ]:
D + D # addition

In [ ]:
D * D # multiplication

In [ ]:
inv(D) # inversion

In [ ]:
sin.(D) # broadcasting

In [ ]:
using LinearAlgebra
eigen(D) # eigensolver

Of course, so far, these operations have suboptimal performance because they don't utilize the special structure inherent to our `DiagonalMatrix` but fall back to generic implementations.

In [ ]:
@which D + D

In [ ]:
D_range = DiagonalMatrix(1:10_000)
@time D_range + D_range

In [ ]:
Base.:+(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag + Db.diag)

In [ ]:
@which D + D

In [ ]:
D_range = DiagonalMatrix(1:10_000)
@time D_range + D_range  # for 10k

D_range = DiagonalMatrix(1:100_000)
@time D_range + D_range;  # for 100k

Important note: **user defined types are just as good as built-in types!**

There is nothing special about built-in types. In fact, [they are implemented in essentially the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Whereas diagonal arrays in something like Numpy are [some C code](https://github.com/numpy/numpy/blob/d4b2d4f80060285ac085ea874aceaf9fa1bfb757/numpy/core/src/multiarray/item_selection.c#L1964) which, regardless of optimisations, could never get close to specialised Julia code.

Let us quickly confirm that our `DiagonalMatrix` type does not come with any performance overhead by benchmarking it in a simple function.

## Consistency of Interfaces

Another benefit of the abstract type/interface/multiple dispatch style is that interfaces are very consistent within Julia, across not just the base language but across packages as well.

An article called [Python vs. Julia: It's also about Consistency](https://towardsdatascience.com/python-vs-julia-its-also-about-consistency-236812dd64ba) shows this point quite well.

## Benchmarking with [`BenchmarkTools.jl`](https://github.com/JuliaCI/BenchmarkTools.jl)

Benchmarking is difficult to do right for many reasons
* computers are noisy machines
* global vs local scope
* the first function call is special in Julia (more later)
* ...

In [ ]:
g(x) = x + 2 * x

In [ ]:
x = rand(2, 2)
@time g.(x)

In [ ]:
function f()
    x = rand(2, 2)
    @time g.(x)
end

In [ ]:
f()

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

General rule: **Don't use `@time` but `@btime`** from [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) and **interpolate (`$`) input arguments**.

In [ ]:
using BenchmarkTools

In [ ]:
@btime g.($x)

In [ ]:
@benchmark g.($x)

### Custom types are just as good as built-in types

Let's compare our custom `DiagonalMatrix` against the standard `Diagonal` type that ships in the `LinearAlgebra` standard library.

In [ ]:
using LinearAlgebra
x = rand(100);
Djl = Diagonal(x)
D = DiagonalMatrix(x)
@btime $Djl + $Djl;
@btime $D + $D;

# Core messages of this notebook

* Duck typing is about **shared behavior** instead of shared structure.
* **User defined types are as good as built-in types.**
* We can **extend Base functions** for our types to implement arithmetics and such.
* **Subtyping an existing interface** can give lots of functionality for free.
* Functions should almost always be benchmarked with **BenchmarkTools.jl's `@btime` and `@benchmark`** instead of `@time`.

# References

- https://github.com/carstenbauer/JuliaHLRS22/blob/main/Day1/2_duck_typing_and_benchmarking.ipynb
- https://towardsdatascience.com/python-vs-julia-its-also-about-consistency-236812dd64ba
- https://docs.julialang.org/en/v1/manual/performance-tips/#Avoid-untyped-global-variables